In [ ]:
%%writefile /.../DE_PKRDC.R


library(tidyverse)
library(monocle)
library(dplyr)
library(Seurat)
options(repr.plot.width = 8, repr.plot.height = 4)


Immunocohort = readRDS(".../Immunocohort.rds")




Immunocohort$Group <- ifelse(grepl("WT", Immunocohort$orig.ident), "WT",
                   ifelse(grepl("PKRDC", Immunocohort$orig.ident, ignore.case = TRUE), "PKRDC", "Other"))




df_cell <- Immunocohort@meta.data
df_gene <- as.data.frame(rownames(Immunocohort@assays$RNA))
colnames(df_gene) <- "gene_name"
gene_count <-Immunocohort@assays$RNA@counts
cds <- cds_construct(UMI = gene_count, df_cell = df_cell, df_gene = df_gene)

fData(cds)$gene_id <- fData(cds)$gene_short_name


cds$Type = "PKRDC"

cds$Type = ifelse(test = cds$Group %in% c("PKRDC"), yes = "PKRDC", no = "WT")


output_folder = "/.../DE_PKRDC/"






find_DE_cluster_regress_out_gene_num <- function (cds, cell_type, core_number) 
{
    cell_type = as.character(cell_type)
    cat("\nCalculate the top expressed cluster of each gene...")
    cds_class_tpm <- class_tpm(cds, cell_type)
    class_label = colnames(cds_class_tpm)
    pData(cds)$tmp = factor(cell_type)
    top_2_cell_type <- do.call(rbind, apply(cds_class_tpm, 1, 
        function(x) {
            a = order(x, decreasing = T)
            b = x[a]
            return(list(max.tissue = class_label[a[1]], second.tissue = class_label[a[2]], 
                max.expr = unname(b[1]), second.expr = unname(b[2]), 
                fold.change = (unname(b[1]) + 0.01)/(unname(b[2]) + 
                  0.01)))
        }))
    top2 <- as.data.frame(top_2_cell_type) %>% mutate(class = max.tissue)
    top2$gene_id = rownames(top_2_cell_type)
    top2$class = as.character(top2$class)
    top2$gene_id = as.character(top2$gene_id)
    cat("\nStart DE analysis for each cluster")
    if (!("num_genes_expressed" %in% names(pData(cds)))) {
        cat("\nData does not have num_genes_expressed column, add detected gene number per cell column...\n")
        cds$num_genes_expressed = Matrix::colSums(exprs(cds) > 
            0)
    }
    DE.genes = differentialGeneTest(cds, fullModelFormulaStr = "~ num_genes_expressed + Type", 
        reducedModelFormulaStr = "~ num_genes_expressed", cores = core_number)
    DE.genes$gene_id = as.character(DE.genes$gene_id)
    DE.genes$gene_short_name = as.character(DE.genes$gene_short_name)
    cat("\n combine the DE genes and top tissues")
    rownames(top2) = top2$gene_id
    top2 = top2 %>% select(-gene_id)
    result = cbind(DE.genes, top2[rownames(DE.genes), ])
    result$qval = (as.numeric(as.character(result$qval)))
    result_2 = lapply(1:length(result), function(x) {
        unlist(result[[x]])
    })
    result_2 = as.data.frame(result_2)
    colnames(result_2) = names(result)
    return(result_2)
}

tmp = unique(cds$Brainregion)
for(i in 1:length(tmp)) {
    cat("\nProcessing sample: ", i, tmp[i])
    output_file = str_c(output_folder, i, ".RDS", sep = "")
    cds_sample = cds[, cds$Brainregion == tmp[i]]
    cds_sample = cds_construct(exprs(cds_sample), pData(cds_sample), fData(cds_sample))
    
    #cds_sample = cds_sample_cell(cds_sample, cds_sample$Type, sample_num = 5000)
    cds_sample = cds_filter_gene_cell_num(cds_sample, 5, 300)
    cds_sample = estimateSizeFactors(cds_sample)
    DE_gene = find_DE_cluster_regress_out_gene_num(cds_sample, cell_type = cds_sample$Type, 20)
    DE_gene$Brainregion = tmp[i]
    saveRDS(DE_gene, file = output_file)
}









In [ ]:
%%writefile /.../DE_PKRDC.sh
#! /bin/bash
#SBATCH --partition=hpc
#SBATCH --ntasks=20
#SBATCH --job-name='DE_PKRDC'
#SBATCH -o /.../DE_PKRDC_s2.out # STDOUT
#SBATCH --error=//DE_PKRDC_s2.out
#SBATCH --mail-user=aabdul@rockefeller.edu # when job is finished, send an email to your address  
#SBATCH --mail-type=ALL

source /rugpfs/fs0/cao_lab/scratch/aabdul/miniconda3/etc/profile.d/conda.sh
conda activate scipy
script_folder='/.../Striatum/DE/'

Rscript $script_folder/DE_PKRDC.R
conda deactivate